In [1]:
# --- CELDA 0: INSTALACIÓN ---
# Instala ultralytics Y fija NumPy a una versión < 2.0 
# para evitar conflictos de entorno en Kaggle.
!pip install ultralytics "numpy<2"

print("¡Bibliotecas 'ultralytics' y 'numpy<2' instaladas!")

¡Bibliotecas 'ultralytics' y 'numpy<2' instaladas!


In [2]:
import os
import shutil
import glob
import random

print("Iniciando la creación y re-partición (80/10/10) del dataset filtrado...")

# --- 1. Definir Rutas ---
INPUT_DIR = '/kaggle/input/graffiti-train-3' 
OUTPUT_DIR = '/kaggle/working/final_dataset_80_10_10'
STAGING_DIR = os.path.join(OUTPUT_DIR, 'staging')
STAGING_IMAGE_DIR = os.path.join(STAGING_DIR, 'images')
STAGING_LABEL_DIR = os.path.join(STAGING_DIR, 'labels')

# --- 2. Definir Mapeo de Clases ---
CLASS_MAPPING = {
    1: 0,  # 'artistico' -> 0
    2: 1   # 'vandalico' -> 1
}
NEW_CLASS_NAMES = ['artistico', 'vandalico']

# --- 3. Limpiar y Crear Directorios ---
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(STAGING_IMAGE_DIR, exist_ok=True)
os.makedirs(STAGING_LABEL_DIR, exist_ok=True)

# --- 4. Procesar y Unificar todos los splits ---
splits_to_process = ['train', 'valid', 'test']
total_files_processed = 0

print(f"\n--- PASO 1: Filtrando y Unificando Clases ---")
for split in splits_to_process:
    print(f"Procesando split de origen: '{split}'...")
    src_label_dir = os.path.join(INPUT_DIR, split, 'labels')
    src_image_dir = os.path.join(INPUT_DIR, split, 'images')
    
    if not os.path.exists(src_label_dir):
        continue

    images_to_keep = set()
    
    for label_file in os.listdir(src_label_dir):
        if not label_file.endswith('.txt'): continue
            
        src_file_path = os.path.join(src_label_dir, label_file)
        dst_file_path = os.path.join(STAGING_LABEL_DIR, label_file)
        
        new_label_lines = []
        try:
            with open(src_file_path, 'r') as f_in:
                lines = f_in.readlines()
            
            for line in lines:
                if not line.strip(): continue
                parts = line.split()
                original_class_id = int(parts[0])
                
                if original_class_id in CLASS_MAPPING:
                    new_class_id = CLASS_MAPPING[original_class_id]
                    new_line = f"{new_class_id} {' '.join(parts[1:])}\n"
                    new_label_lines.append(new_line)
            
            if new_label_lines:
                with open(dst_file_path, 'w') as f_out:
                    f_out.writelines(new_label_lines)
                images_to_keep.add(os.path.splitext(label_file)[0])

        except Exception as e:
            print(f"  Error procesando {label_file}: {e}")

    copied_count = 0
    if os.path.exists(src_image_dir):
        for img_file in os.listdir(src_image_dir):
            base_name = os.path.splitext(img_file)[0]
            if base_name in images_to_keep:
                src_img_path = os.path.join(src_image_dir, img_file)
                dst_img_path = os.path.join(STAGING_IMAGE_DIR, img_file)
                shutil.copy2(src_img_path, dst_img_path)
                copied_count += 1
            
    print(f"  Se copiaron {copied_count} imágenes a staging.")
    total_files_processed += copied_count

# --- 6. Re-partición 80/10/10 ---
print("\n--- PASO 2: Re-partiendo el dataset en 80/10/10 ---")
all_image_paths = glob.glob(os.path.join(STAGING_IMAGE_DIR, '*.*'))
random.seed(42)
random.shuffle(all_image_paths)

total_count = len(all_image_paths)
train_end = int(total_count * 0.8)
valid_end = train_end + int(total_count * 0.1)

splits_files = {
    'train': all_image_paths[:train_end],
    'valid': all_image_paths[train_end:valid_end],
    'test': all_image_paths[valid_end:]
}

for split_name, image_files in splits_files.items():
    print(f"Creando split '{split_name}' con {len(image_files)} imágenes...")
    final_img_dir = os.path.join(OUTPUT_DIR, split_name, 'images')
    final_label_dir = os.path.join(OUTPUT_DIR, split_name, 'labels')
    os.makedirs(final_img_dir, exist_ok=True)
    os.makedirs(final_label_dir, exist_ok=True)
    
    for img_path in image_files:
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        label_file_name = base_name + '.txt'
        src_label_path = os.path.join(STAGING_LABEL_DIR, label_file_name)
        dst_label_path = os.path.join(final_label_dir, label_file_name)
        
        shutil.move(img_path, final_img_dir)
        if os.path.exists(src_label_path):
            shutil.move(src_label_path, dst_label_path)

shutil.rmtree(STAGING_DIR)

print("\n¡Proceso de filtrado y re-partición (80/10/10) completado!")

Iniciando la creación y re-partición (80/10/10) del dataset filtrado...

--- PASO 1: Filtrando y Unificando Clases ---
Procesando split de origen: 'train'...
Procesando split de origen: 'valid'...
Procesando split de origen: 'test'...

--- PASO 2: Re-partiendo el dataset en 80/10/10 ---
Creando split 'train' con 0 imágenes...
Creando split 'valid' con 0 imágenes...
Creando split 'test' con 0 imágenes...

¡Proceso de filtrado y re-partición (80/10/10) completado!


In [3]:
import os
import yaml

print("Creando el archivo YAML para el dataset 80/10/10...")

DATASET_80_10_10_DIR = '/kaggle/working/final_dataset_80_10_10'
YAML_PATH = '/kaggle/working/data_final_80_10_10.yaml'

yaml_data = {
    'train': os.path.join(DATASET_80_10_10_DIR, 'train/images'),
    'val': os.path.join(DATASET_80_10_10_DIR, 'valid/images'),
    'test': os.path.join(DATASET_80_10_10_DIR, 'test/images'),
    'nc': 2,
    'names': ['artistico', 'vandalico']
}

with open(YAML_PATH, 'w') as f:
    yaml.dump(yaml_data, f, sort_keys=False)

print(f"Archivo YAML creado exitosamente en: {YAML_PATH}")

Creando el archivo YAML para el dataset 80/10/10...
Archivo YAML creado exitosamente en: /kaggle/working/data_final_80_10_10.yaml


In [4]:
from ultralytics import YOLO
import os

print("¡Hola! Iniciando el script de entrenamiento con YOLOv8s...")

YAML_PARA_ENTRENAR = '/kaggle/working/data_final_80_10_10.yaml'
OUTPUT_DIR = '/kaggle/working/'
TRAIN_RUN_NAME = 'artistico_vandalico_train'
TEST_RUN_NAME = 'artistico_vandalico_test'

print("Cargando el modelo YOLOv8s...")
model = YOLO('yolov8s.pt')

print("Iniciando el entrenamiento...")
rresults = model.train(
    data=YAML_PARA_ENTRENAR,
    imgsz=640,          # Tamaño estándar, ideal para SAHI después
    batch=32,           # 32 aprovecha bien la T4. Si falla, baja a 16.
    project=OUTPUT_DIR,
    name=TRAIN_RUN_NAME,
    exist_ok=True,
    
    # --- HIPERPARÁMETROS CLAVE ---
    epochs=3,       # Damos tiempo suficiente para aprender
    patience=20,        # Si no mejora en 20 épocas, paramos (ahorra tiempo)
    optimizer='auto',   # Ultralytics elegirá SGD o AdamW automáticamente
    
    # --- OPTIMIZACIONES DE GPU ---
    device=0,           # Forzamos explícitamente la GPU 0
    workers=8,          # Núcleos de CPU para cargar datos rápido hacia la GPU
    amp=True,           # Precisión mixta (más rápido)
    
    # --- AUGMENTATION (Opcional: ayuda si tienes pocas fotos) ---
    degrees=10.0,       # Rotar levemente (los grafitis no siempre están rectos)
    fliplr=0.5,         # Espejo horizontal (un grafiti al revés sigue siendo grafiti)
    mosaic=1.0,         # Fundamental para objetos en contexto
)

print("\n¡Entrenamiento completado!")

# --- Evaluación en Test ---
print("\nCargando el MEJOR modelo para la evaluación en 'test'...")
best_model_path = os.path.join(OUTPUT_DIR, TRAIN_RUN_NAME, 'weights/best.pt')

if os.path.exists(best_model_path):
    best_model = YOLO(best_model_path)
    print("Ejecutando evaluación en el set de 'test'...")
    test_results = best_model.val(
        data=YAML_PARA_ENTRENAR,
        split='test',
        project=OUTPUT_DIR,
        name=TEST_RUN_NAME
    )
    print("\n¡Evaluación de test completada!")
else:
    print("No se encontró el modelo entrenado.")

¡Hola! Iniciando el script de entrenamiento con YOLOv8s...
Cargando el modelo YOLOv8s...
Iniciando el entrenamiento...
Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data_final_80_10_10.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=artistico_v

FileNotFoundError: [34m[1mtrain: [0mError loading data from /kaggle/working/final_dataset_80_10_10/train/images
See https://docs.ultralytics.com/datasets for dataset formatting guidance.